In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
#Visualizate fist columns
data.head()

In [ ]:
#Checking for general data an non-nulls
data.info()

In [ ]:
#Filling NA from "bmi" column with the mean of the column
data["bmi"] = data["bmi"].fillna(data["bmi"].mean())

In [ ]:
#Values from categories

print(data["gender"].unique())
print(data["ever_married"].unique())
print(data["work_type"].unique())
print(data["Residence_type"].unique())
print(data["smoking_status"].unique())

In [ ]:
#Making the replacement of the categorical variables

#Gender column
data["gender"] = data["gender"].replace(["Male"], 0)
data["gender"] = data["gender"].replace(["Female"], 1)
data["gender"] = data["gender"].replace(["Other"], 2)

#Ever married column
data["ever_married"] = data["ever_married"].replace(["Yes"], 0)
data["ever_married"] = data["ever_married"].replace(["No"], 1)

#work_type column
data["work_type"] = data["work_type"].replace(["Private"], 0)
data["work_type"] = data["work_type"].replace(["Self-employed"], 1)
data["work_type"] = data["work_type"].replace(["Govt_job"], 2)
data["work_type"] = data["work_type"].replace(["children"], 3)
data["work_type"] = data["work_type"].replace(["Never_worked"], 4)

#residence_type column
data["Residence_type"] = data["Residence_type"].replace(["Urban"], 0)
data["Residence_type"] = data["Residence_type"].replace(["Rural"], 1)

#smoking_status
data["smoking_status"] = data["smoking_status"].replace(["formerly smoked"], 0)
data["smoking_status"] = data["smoking_status"].replace(["never smoked"], 1)
data["smoking_status"] = data["smoking_status"].replace(["smokes"], 2)
data["smoking_status"] = data["smoking_status"].replace(["Unknown"], 3)

In [ ]:
#Defining prectors variables and target
#For predictors i will use all de columns and the target value are de strokes

predictors = data.columns[1:11]
target = data.columns[11]

X = data[predictors]
Y = data[target]

In [ ]:
def rand_forest(criterion="entropy", n_estimators=500, min_samples_split=5, threshold=0.5):
    
    #importing the libraries
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    import seaborn as sns
    from sklearn import metrics
    import matplotlib.pyplot as plt

    forest_class = RandomForestClassifier(criterion=criterion, n_jobs=2, oob_score=True, 
                                             n_estimators=n_estimators, min_samples_split=min_samples_split)
    forest_class.fit(X,Y)

    pred = forest_class.predict(X)

    prob = forest_class.predict_proba(X)

    #Defining the threshold to decide the probability limit for forecasting an stroke
    prob = prob[:, 1]
    prob_df = pd.DataFrame(prob)
    prob_df["prediction"] = np.where(prob_df[0] >= threshold, 1, 0 )

    matrix = confusion_matrix(Y, prob_df.prediction)
    matrix


    fig, g = plt.subplots(figsize=(5,5))
    g = sns.heatmap(matrix.T, annot=True, cbar= False, fmt="",
                xticklabels=["0", "1"], 
                yticklabels=["0", "1"])
    g.set_title("Confusion Matrix")
    g.set_xlabel("Reals")
    g.set_ylabel("Predictions")


    print(classification_report(Y, prob_df.prediction, target_names= ["0","1"]))
    
    #Making a cross validation with 10 splits
    
    from sklearn.model_selection import KFold, cross_val_score
    cv = KFold(n_splits= 10, shuffle=True, random_state=1)
    score = np.mean(cross_val_score(forest_class, X, Y, scoring = "accuracy", cv= cv))
    
    print("--------------------------------------")
    
    print("The Cross Validation with 10 samples got an  accuracy score of {}%".format(round(score*100,2)))

In [ ]:
rand_forest(threshold=0.4)
#Using a theshold of 0.45 to catalog an stroke

The Random Forest model adjusted very well to the data, having a 100% of accuracy. In my opinion is very important to have good accuracy predicting the strokes, even if in the effort of increasing that acc the model predicts non strokes as strokes. For checking if the model doesn´t overfit the data, i made a cross validation taking 10 splits. In that cross validation the model have an 95% of accuracy.

This is my first publish, any suggestiongs ar well welcome!!